# Paralell Pandas

Increase the speed of the calculation of time series features by chunking the dataframe and using paralell processing with Pythons `multiprocessing` library.

In [1]:
import numpy as np
import pandas as pd


The real dataset I am working on is a set of daily satellite measurements (from Copernicus Sentinel-1) ranging from ca. -25 to 0.

In [2]:
ts_df = pd.DataFrame(np.random.random(size=(365, 3000)))


In [3]:
ts_df.head()


,0,1,2,3,4,5,6,7,8,9,...,2990,2991,2992,2993,2994,2995,2996,2997,2998,2999
0,0.797705,0.192938,0.598928,0.049489,0.426070,0.993818,0.234540,0.487642,0.280514,0.270824,...,0.625908,0.274562,0.572727,0.483228,0.947370,0.470938,0.034136,0.862813,0.624245,0.225501
1,0.962685,0.329267,0.430161,0.077733,0.789927,0.648647,0.688537,0.203000,0.063319,0.739370,...,0.954632,0.434089,0.272374,0.136659,0.516720,0.813053,0.876236,0.732918,0.179523,0.974357
2,0.946652,0.008486,0.637859,0.902016,0.098176,0.415215,0.802620,0.940340,0.028065,0.330874,...,0.267486,0.882917,0.213091,0.100764,0.113346,0.797444,0.501101,0.542769,0.856774,0.200036
3,0.160855,0.028498,0.239031,0.904416,0.422833,0.543515,0.111439,0.469557,0.054797,0.447173,...,0.431720,0.430018,0.578983,0.237469,0.769015,0.280197,0.176195,0.191841,0.113176,0.545080
4,0.986059,0.843225,0.527381,0.538054,0.798150,0.872702,0.775131,0.926873,0.899077,0.940833,...,0.550767,0.497773,0.216634,0.261508,0.964396,0.512830,0.652238,0.166521,0.838778,0.977973


I want to calculate a number of temporal features to be used as input for a regression analysis. These will be calculated for each column. The features themselves are straightforward multi-temporal features such as percentiles, using a lagged time series and some based on Fourier transformation.

In [4]:
def feature_calculation(df):
    # create DataFrame and populate with stdDev
    result = pd.DataFrame(df.std(axis=0))
    result.columns = ["stdDev"]

    # mean
    result["mean"] = df.mean(axis=0)

    # percentiles
    for i in [0.1, 0.25, 0.5, 0.75, 0.9]:
        result[str(int(i*100)) + "perc"] = df.quantile(q=i)

    # percentile differences / amplitudes
    result["diff_90perc10perc"] = (result["10perc"] - result["90perc"])
    result["diff_75perc25perc"] = (result["75perc"] - result["25perc"])

    # percentiles of lagged time-series
    for lag in [10, 20, 30, 40, 50]:
        for i in [0.1, 0.25, 0.5, 0.75, 0.9]:
            result["lag" + str(lag) + "_" + str(int(i*100)) + "perc"] = (df - df.shift(lag)).quantile(q=i)

    # fft
    df_fft = np.fft.fft(df, axis=0)  # fourier transform only along time axis
    result["fft_angle_mean"] = np.mean(np.angle(df_fft, deg=True), axis=0)
    result["fft_angle_min"] = np.min(np.angle(df_fft, deg=True), axis=0)
    result["fft_angle_max"] = np.max(np.angle(df_fft, deg=True), axis=0)

    return result


Testing how long the calculation takes for a small test dataset.

In [13]:
%%timeit -n 3 -r 3
ts_features = feature_calculation(ts_df)


11.4 s ± 86.3 ms per loop (mean ± std. dev. of 3 runs, 3 loops each)


The calculation takes quite some time and increases linear with the number of columns. My real dataset has more than 700k columns instead of the 3000 we use here.

During the calculation only one core is used. As the calculation is performed for each column we can split the dataframe into a number of subsets and utulize multiple cores to calculate the features - making this an embarassingly paralell problem.

In [6]:
from multiprocessing import Pool

def paralell_feature_calculation(df, partitions=10, processes=4):
    # calculate features in paralell by splitting the dataframe into partitions and using paralell processes

    pool = Pool(processes)

    df_split = np.array_split(df, partitions, axis=1)  # split dataframe into partitions column wise

    df = pd.concat(pool.map(feature_calculation, df_split))
    pool.close()
    pool.join()

    return df


In [18]:
%%timeit -n 3 -r 3
ts_features_paralell = paralell_feature_calculation(ts_df, partitions=14, processes=7)


2.06 s ± 15.4 ms per loop (mean ± std. dev. of 3 runs, 3 loops each)


Compare the two results to make sure we get identical results using both feature calculation functions.

In [12]:
ts_features.equals(ts_features_paralell)


True

Using a simple paralellization routine the time series features are now calculated about 5 times faster - a significant time saving when working with large dataframes.